This notebook is designed to make light curves using the same technique as the PTF summer school, so that a training set of sources that are QSOs and RRL can be included in the model for the pre-selected field of interest. 

In [1]:
from __future__ import division
import numpy as np
import astropy.io.fits as fits
import astropy.coordinates as coords
import astropy.units as u
from astropy.time import Time
import astropy.utils
from glob import glob
import matplotlib.pyplot as plt
import shelve, pickle
import uuid
from IPython.display import HTML, Javascript, display
import time
import FATS
%matplotlib inline

In [65]:
# START with field 4163

reference_catalog = '../data/other_fields/4163/PTF_d004163_f02_c09_u000152621_p12_sexcat.ctlg'
# select R-band data (f02)
epoch_catalogs = glob('../data/other_fields/4163/PTF_2*f02*.ctlg')

In [3]:
def load_ref_catalog(reference_catalog):
    hdus = fits.open(reference_catalog)
    data = hdus[1].data
    # filter flagged detections
    w = ((data['flags'] & 506 == 0) & (data['MAG_AUTO'] < 99))
    data = data[w]

    ref_coords = coords.SkyCoord(data['X_WORLD'], data['Y_WORLD'],frame='icrs',unit='deg')

    star_class = np.array(data["CLASS_STAR"]).T
    
    return np.vstack([data['MAG_AUTO'],data['MAGERR_AUTO']]).T, ref_coords, star_class

In [9]:
ref_mags, ref_coords, star_class = load_ref_catalog(reference_catalog)

print "There are %s sources in the reference image" % len(ref_mags)
print "..."
print "There are %s epochs for this field" % len(epoch_catalogs)

There are 11094 sources in the reference image
...
There are 116 epochs for this field


In [5]:
def crossmatch_epochs(reference_coords, epoch_catalogs):
    
    n_stars = len(reference_coords)
    n_epochs = len(epoch_catalogs)
    
    mags = np.ma.zeros([n_stars, n_epochs])
    magerrs = np.ma.zeros([n_stars, n_epochs])
    mjds = np.ma.zeros(n_epochs)
    
    with astropy.utils.console.ProgressBar(len(epoch_catalogs),ipython_widget=True) as bar:
        for i, catalog in enumerate(epoch_catalogs):
            hdus = fits.open(catalog)
            data = hdus[1].data
            hdr = hdus[2].header
            # filter flagged detections
            w = ((data['flags'] & 506 == 0) & (data['imaflags_iso'] & 1821 == 0))
            data = data[w]

            epoch_coords = coords.SkyCoord(data['X_WORLD'], data['Y_WORLD'],frame='icrs',unit='deg')
            idx, sep, dist = coords.match_coordinates_sky(epoch_coords, reference_coords)
        
            wmatch = (sep <= 1.5*u.arcsec)
        
        # store data
            if np.sum(wmatch):
                mags[idx[wmatch],i] = data[wmatch]['MAG_APER'][:,2] + data[wmatch]['ZEROPOINT']
                magerrs[idx[wmatch],i] = data[wmatch]['MAGERR_APER'][:,2]
                mjds[i] = hdr['OBSMJD']

            bar.update()
    return mjds, mags, magerrs

In [10]:
mjds,mags,magerrs = crossmatch_epochs(ref_coords, epoch_catalogs)

In [11]:
wbad = (mags < 10) | (mags > 25)
mags[wbad] = np.ma.masked
magerrs[wbad] = np.ma.masked

In [14]:
def relative_photometry(ref_mags, star_class, mags, magerrs):
    #make copies, as we're going to modify the masks
    all_mags = mags.copy()
    all_errs = magerrs.copy()
    
    # average over observations
#     medmags = np.ma.median(all_mags,axis=1)  # use the mag in the reference image
    refmags = np.ma.array(ref_mags[:,0])
#     stdmags = np.ma.std(all_mags,axis=1)     # use outlier resistant median absolute deviation
    madmags = 1.48*np.ma.median(np.abs(all_mags - np.ma.median(all_mags, axis = 1).reshape(len(ref_mags),1)), axis = 1)
    MSE = np.ma.mean(all_errs**2.,axis=1)

    # exclude bad stars: highly variable, saturated, or faint
    # use excess variance to find bad objects
    excess_variance = madmags**2. - MSE
    wbad = np.where((np.abs(excess_variance) > 0.1) | (refmags < 14.5) | (refmags > 17) | (star_class < 0.9))
    # mask them out
    refmags[wbad] = np.ma.masked
    
    # exclude stars that are not detected in a majority of epochs
    Nepochs = len(all_mags[0,:])
    nbad = np.where(np.ma.sum(all_mags > 1, axis = 1) <= Nepochs/2.)
    refmags[nbad] = np.ma.masked

    # for each observation, take the median of the difference between the median mag and the observed mag
    # annoying dimension swapping to get the 1D vector to blow up right
    relative_zp = np.ma.median(all_mags - refmags.reshape((len(all_mags),1)),axis=0)

    return relative_zp

In [15]:
# compute the relative photometry and subtract it. Don't fret about error propagation
rel_zp = relative_photometry(ref_mags, star_class, mags, magerrs)
mags -= np.ma.resize(rel_zp, mags.shape)

In [16]:
outfile = reference_catalog.split('/')[-1].replace('ctlg','shlv')
shelf = shelve.open('../data/'+outfile,flag='c',protocol=pickle.HIGHEST_PROTOCOL)
shelf['mjds'] = mjds
shelf['mags'] = mags
shelf['magerrs'] = magerrs
shelf['ref_coords'] = ref_coords
shelf.close()

In [17]:
def source_lightcurve(rel_phot_shlv, ra, dec, matchr = 1.0):
    """Crossmatch ra and dec to a PTF shelve file, to return light curve of a given star"""
    shelf = shelve.open(rel_phot_shlv)
    ref_coords = coords.SkyCoord(shelf["ref_coords"].ra, shelf["ref_coords"].dec,frame='icrs',unit='deg')    
    
    source_coords = coords.SkyCoord(ra, dec,frame='icrs',unit='deg')
    idx, sep, dist = coords.match_coordinates_sky(source_coords, ref_coords)        
    
    wmatch = (sep <= matchr*u.arcsec)
    
    if sum(wmatch) == 1:
        mjds = shelf["mjds"]
        mags = shelf["mags"][idx]
        magerrs = shelf["magerrs"][idx]
        
        return mjds, mags, magerrs

    else:
        return "There are no matches to the provided coordinates within %.1f arcsec" % (matchr)

In [124]:
reference_catalog = '../data/other_fields/4163/PTF_d004163_f02_c09_u000152621_p12_sexcat.ctlg'
outfile = reference_catalog.split('/')[-1].replace('ctlg','shlv')

ra = np.array([253.177886, 252.830368, 253.063609, 253.286147])
dec = np.array([32.266276, 32.02584, 31.901347, 32.535967])
RRLfeats = []
for r, d in zip(ra, dec):
    source_mjds, source_mags, source_magerrs = source_lightcurve('../data/'+outfile, r, d)
    [mag, time, error] = FATS.Preprocess_LC(source_mags, source_mjds, source_magerrs).Preprocess()

    lc = np.array([mag, time, error])
    feats = FATS.FeatureSpace(Data=['magnitude', 'time', 'error']).calculateFeature(lc)
    feat_row = np.reshape(feats.result(method='array'), (1,59))
        
    if len(RRLfeats) == 0:
        RRLfeats = feat_row
    else:
        RRLfeats = np.append(RRLfeats, feat_row, axis = 0)

In [125]:
reference_catalog = '../data/other_fields/4163/PTF_d004163_f02_c09_u000152621_p12_sexcat.ctlg'
outfile = reference_catalog.split('/')[-1].replace('ctlg','shlv')

ra = np.array([252.97529, 252.92493,252.97529, 252.88804, 253.33945, 253.10062, 253.3395, 253.4526])
dec = np.array([31.551398, 32.244859, 31.551391,31.707791, 31.538562,32.017281,31.538542,31.518162])
QSOfeats = []
for r, d in zip(ra, dec):
    source_mjds, source_mags, source_magerrs = source_lightcurve('../data/'+outfile, r, d)
    [mag, time, error] = FATS.Preprocess_LC(source_mags, source_mjds, source_magerrs).Preprocess()

    lc = np.array([mag, time, error])
    feats = FATS.FeatureSpace(Data=['magnitude', 'time', 'error']).calculateFeature(lc)
    feat_row = np.reshape(feats.result(method='array'), (1,59))
        
    if len(QSOfeats) == 0:
        QSOfeats = feat_row
    else:
        QSOfeats = np.append(QSOfeats, feat_row, axis = 0)

In [111]:
ra, dec = 253.4526,31.518162
shelf = shelve.open('../data/'+outfile)
ref_coords = coords.SkyCoord(shelf["ref_coords"].ra, shelf["ref_coords"].dec,frame='icrs',unit='deg')    
    
source_coords = coords.SkyCoord(ra, dec,frame='icrs',unit='deg')
idx, sep, dist = coords.match_coordinates_sky(source_coords, ref_coords)        
    
wmatch = (sep <= 1*u.arcsec)

wmatch

array([ True], dtype=bool)

In [28]:
## 2nd with field 3696

reference_catalog = '../data/other_fields/3696/PTF_d003696_f02_c06_u000154869_p12_sexcat.ctlg'
# select R-band data (f02)
epoch_catalogs = glob('../data/other_fields/3696/PTF_2*f02*.ctlg')

ref_mags, ref_coords, star_class = load_ref_catalog(reference_catalog)

print "There are %s sources in the reference image" % len(ref_mags)
print "..."
print "There are %s epochs for this field" % len(epoch_catalogs)

mjds,mags,magerrs = crossmatch_epochs(ref_coords, epoch_catalogs)

wbad = (mags < 10) | (mags > 25)
mags[wbad] = np.ma.masked
magerrs[wbad] = np.ma.masked

# compute the relative photometry and subtract it. Don't fret about error propagation
rel_zp = relative_photometry(ref_mags, star_class, mags, magerrs)
mags -= np.ma.resize(rel_zp, mags.shape)

There are 12427 sources in the reference image
...
There are 120 epochs for this field



In [29]:
outfile = reference_catalog.split('/')[-1].replace('ctlg','shlv')
shelf = shelve.open('../data/'+outfile,flag='c',protocol=pickle.HIGHEST_PROTOCOL)
shelf['mjds'] = mjds
shelf['mags'] = mags
shelf['magerrs'] = magerrs
shelf['ref_coords'] = ref_coords
shelf.close()

In [126]:
reference_catalog = '../data/other_fields/3696/PTF_d003696_f02_c06_u000154869_p12_sexcat.ctlg'
outfile = reference_catalog.split('/')[-1].replace('ctlg','shlv')

ra = np.array([253.65622, 253.589594])
dec = np.array([20.743137, 20.696087])
for r, d in zip(ra, dec):
    source_mjds, source_mags, source_magerrs = source_lightcurve('../data/'+outfile, r, d)
    [mag, time, error] = FATS.Preprocess_LC(source_mags, source_mjds, source_magerrs).Preprocess()

    lc = np.array([mag, time, error])
    feats = FATS.FeatureSpace(Data=['magnitude', 'time', 'error']).calculateFeature(lc)
    feat_row = np.reshape(feats.result(method='array'), (1,59))
        
    if len(RRLfeats) == 0:
        RRLfeats = feat_row
    else:
        RRLfeats = np.append(RRLfeats, feat_row, axis = 0)

In [127]:
reference_catalog = '../data/other_fields/3696/PTF_d003696_f02_c06_u000154869_p12_sexcat.ctlg'
outfile = reference_catalog.split('/')[-1].replace('ctlg','shlv')

ra = np.array([253.47151, 253.50356, 253.50356, 253.59031, 253.6131,  253.78818, 253.86388, 253.86388, 253.91854, 253.92526])
dec = np.array([20.346592, 21.149149, 21.149149, 20.494092, 20.564037,  21.322201, 20.663265, 20.66327, 20.622025, 20.651342])
for r, d in zip(ra, dec):
    source_mjds, source_mags, source_magerrs = source_lightcurve('../data/'+outfile, r, d)
    [mag, time, error] = FATS.Preprocess_LC(source_mags, source_mjds, source_magerrs).Preprocess()

    lc = np.array([mag, time, error])
    feats = FATS.FeatureSpace(Data=['magnitude', 'time', 'error']).calculateFeature(lc)
    feat_row = np.reshape(feats.result(method='array'), (1,59))
        
    if len(QSOfeats) == 0:
        QSOfeats = feat_row
    else:
        QSOfeats = np.append(QSOfeats, feat_row, axis = 0)

In [70]:
## 3rd with field 3696

reference_catalog = '../data/other_fields/22682/PTF_d022682_f02_c11_u000096411_p12_sexcat.ctlg'
# select R-band data (f02)
epoch_catalogs = glob('../data/other_fields/22682/PTF_2*f02*.ctlg')

ref_mags, ref_coords, star_class = load_ref_catalog(reference_catalog)

print "There are %s sources in the reference image" % len(ref_mags)
print "..."
print "There are %s epochs for this field" % len(epoch_catalogs)

mjds,mags,magerrs = crossmatch_epochs(ref_coords, epoch_catalogs)

wbad = (mags < 10) | (mags > 25)
mags[wbad] = np.ma.masked
magerrs[wbad] = np.ma.masked

# compute the relative photometry and subtract it. Don't fret about error propagation
rel_zp = relative_photometry(ref_mags, star_class, mags, magerrs)
mags -= np.ma.resize(rel_zp, mags.shape)

There are 14498 sources in the reference image
...
There are 73 epochs for this field



In [71]:
outfile = reference_catalog.split('/')[-1].replace('ctlg','shlv')
shelf = shelve.open('../data/'+outfile,flag='c',protocol=pickle.HIGHEST_PROTOCOL)
shelf['mjds'] = mjds
shelf['mags'] = mags
shelf['magerrs'] = magerrs
shelf['ref_coords'] = ref_coords
shelf.close()

In [128]:
reference_catalog = '../data/other_fields/22682/PTF_d022682_f02_c11_u000096411_p12_sexcat.ctlg'
outfile = reference_catalog.split('/')[-1].replace('ctlg','shlv')

ra = np.array([311.527209, 311.546485, 311.711246])
dec = np.array([-0.133972, -0.042581, -0.898823])
for r, d in zip(ra, dec):
    source_mjds, source_mags, source_magerrs = source_lightcurve('../data/'+outfile, r, d)
    [mag, time, error] = FATS.Preprocess_LC(source_mags, source_mjds, source_magerrs).Preprocess()

    lc = np.array([mag, time, error])
    feats = FATS.FeatureSpace(Data=['magnitude', 'time', 'error']).calculateFeature(lc)
    feat_row = np.reshape(feats.result(method='array'), (1,59))
        
    if len(RRLfeats) == 0:
        RRLfeats = feat_row
    else:
        RRLfeats = np.append(RRLfeats, feat_row, axis = 0)

In [129]:
reference_catalog = '../data/other_fields/22682/PTF_d022682_f02_c11_u000096411_p12_sexcat.ctlg'
outfile = reference_catalog.split('/')[-1].replace('ctlg','shlv')

ra = np.array([311.54785, 311.59198, 311.81836])
dec = np.array([-0.60713278, -0.48408186, -0.28332211])
for r, d in zip(ra, dec):
    source_mjds, source_mags, source_magerrs = source_lightcurve('../data/'+outfile, r, d)
    [mag, time, error] = FATS.Preprocess_LC(source_mags, source_mjds, source_magerrs).Preprocess()

    lc = np.array([mag, time, error])
    feats = FATS.FeatureSpace(Data=['magnitude', 'time', 'error']).calculateFeature(lc)
    feat_row = np.reshape(feats.result(method='array'), (1,59))
        
    if len(QSOfeats) == 0:
        QSOfeats = feat_row
    else:
        QSOfeats = np.append(QSOfeats, feat_row, axis = 0)

In [146]:
STARfeats = []
for mags, magerrs in zip(shelf['mags'], shelf['magerrs']):
    if (sum(mags.mask) - len(mags)) > -20 or np.ma.median(mags) > 19.5:
        continue
    else:
        lc_mag = mags
        lc_mjd = shelf['mjds']
        lc_magerr = magerrs

        [mag, time, error] = FATS.Preprocess_LC(lc_mag, lc_mjd, lc_magerr).Preprocess()

        lc = np.array([mag, time, error])
        feats = FATS.FeatureSpace(Data=['magnitude', 'time', 'error']).calculateFeature(lc)
        feat_row = np.reshape(feats.result(method='array'), (1,59))
        
        if len(STARfeats) == 0:
            STARfeats = feat_row
        elif len(STARfeats) < 100:
            STARfeats = np.append(STARfeats, feat_row, axis = 0)
        else:
            break

CAR lik se fue a inf
CAR lik se fue a inf
CAR lik se fue a inf
CAR lik se fue a inf
CAR lik se fue a inf


In [147]:
TSfeats = np.append(STARfeats, QSOfeats, axis = 0)
TSfeats = np.append(TSfeats, RRLfeats, axis = 0)

TSlabels = np.empty(len(TSfeats), dtype = '|S4')
TSlabels[0:len(STARfeats)] = 'star'
TSlabels[len(STARfeats):len(STARfeats)+len(QSOfeats)] = 'qso'
TSlabels[-len(RRLfeats):] = 'rrl'

from astropy.table import Table
feat_table = Table(TSfeats, names = tuple(feats.result(method='features')))
feat_table['class'] = TSlabels
feat_table.write('../data/TS_PTF_feats.csv', format='csv')

In [148]:
feat_table

Amplitude,AndersonDarling,Autocor_length,Beyond1Std,CAR_mean,CAR_sigma,CAR_tau,Con,Eta_e,FluxPercentileRatioMid20,FluxPercentileRatioMid35,FluxPercentileRatioMid50,FluxPercentileRatioMid65,FluxPercentileRatioMid80,Freq1_harmonics_amplitude_0,Freq1_harmonics_amplitude_1,Freq1_harmonics_amplitude_2,Freq1_harmonics_amplitude_3,Freq1_harmonics_rel_phase_0,Freq1_harmonics_rel_phase_1,Freq1_harmonics_rel_phase_2,Freq1_harmonics_rel_phase_3,Freq2_harmonics_amplitude_0,Freq2_harmonics_amplitude_1,Freq2_harmonics_amplitude_2,Freq2_harmonics_amplitude_3,Freq2_harmonics_rel_phase_0,Freq2_harmonics_rel_phase_1,Freq2_harmonics_rel_phase_2,Freq2_harmonics_rel_phase_3,Freq3_harmonics_amplitude_0,Freq3_harmonics_amplitude_1,Freq3_harmonics_amplitude_2,Freq3_harmonics_amplitude_3,Freq3_harmonics_rel_phase_0,Freq3_harmonics_rel_phase_1,Freq3_harmonics_rel_phase_2,Freq3_harmonics_rel_phase_3,LinearTrend,MaxSlope,Mean,Meanvariance,MedianAbsDev,MedianBRP,PairSlopeTrend,PercentAmplitude,PercentDifferenceFluxPercentile,PeriodLS,Period_fit,Psi_CS,Psi_eta,Q31,Rcs,Skew,SlottedA_length,SmallKurtosis,Std,StetsonK,StetsonK_AC,class
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,string32
0.197540044785,0.908594449856,1.0,0.279069767442,302.819377892,-0.272564669582,0.0641990449674,0.0,1907.69935531,0.0614505476304,0.182309218435,0.271365988578,0.516850306242,0.700908434348,0.0843349165462,0.0353664749469,0.00312533086886,0.0316351212829,0.0,0.783685978511,-1.18290953576,-0.71032481131,0.0734110451595,0.0368016939867,0.0118545910066,0.0337126729402,0.0,-0.437450379065,-0.283168202881,-1.6599340106,0.0486391239807,0.0222567777027,0.0211188293315,0.00801741133962,0.0,-1.69525245355,-0.445405738266,-1.68146969149,-0.000799303952283,9.34605744136,19.4407148583,0.00545397237887,0.0529227256775,0.46511627907,0.1,0.0150250992626,0.0201374619312,0.151940506879,0.981874273874,0.285694562945,1.47256978236,0.102793455124,0.198573136743,0.487843520704,0.0272999999943,0.664855443763,0.106029121863,0.788610005187,0.703502973318,star
0.113285422325,0.96751347026,1.0,0.289473684211,10.9950473138,0.0256210524803,1.67434142432,0.0,4589.17655481,0.0574002836648,0.141577298654,0.208940542636,0.380772017183,0.615748497495,0.048147220541,0.0189428578339,0.0192185645779,0.0268807049831,0.0,1.20349053958,0.671201361407,1.9540749226,0.0350774728386,0.0070999791073,0.0107857881215,0.0241921203726,0.0,0.408443138941,0.116748721558,0.659922973429,0.0201703613555,0.00451064805712,0.00664032367557,0.00412670701429,0.0,-1.26720574185,-0.132114353605,-1.33678251105,-0.000628581225145,7.78863174759,18.4094631797,0.00285719494377,0.0223021507263,0.526315789474,0.1,0.00798912925435,0.0126930793013,0.0443661782429,0.918996892291,0.228592058536,1.18339781269,0.0448484420776,0.127987160457,0.477387045134,0.0272999999943,1.23439455637,0.0525994251148,0.753076812314,0.734013867147,star
0.182755470276,0.960426631425,1.0,0.288888888889,32.6483692197,1.7050992146e-09,0.594603177221,0.0,1879.13765129,0.0968609060638,0.170971641148,0.305798900562,0.491453729399,0.74372201608,0.0750223370121,0.0293680478991,0.0327066007333,0.0494954082913,0.0,-0.347046869065,-0.537439314018,1.58752642841,0.0709648603768,0.00853801223506,0.017984024217,0.0364179031894,0.0,0.912554530513,2.70211084278,2.18987553727,0.041993892312,0.0166846218298,0.0156986235913,0.0164459245386,0.0,-2.02839596713,0.295106431859,-2.14799302999,0.00112880539105,7.95855877524,19.4128240691,0.00482470977507,0.0466747283936,0.488888888889,-0.1,0.0110965470384,0.0180529127491,0.0328320826162,0.97367489938,0.186756841715,1.89199

In [145]:
len(QSOfeats)

21